In [8]:
import numpy as np
from qiskit import Aer
from qiskit.compiler import assemble
from qiskit.ignis.verification.tomography import GatesetTomographyFitter
from qiskit.ignis.verification.tomography import gateset_tomography_circuits
from qiskit.ignis.verification.tomography.basis import default_gateset_basis

from qiskit.providers.aer.noise.errors import QuantumError
from qiskit.providers.aer.noise.errors import depolarizing_error
from qiskit.providers.aer.noise import NoiseModel

from qiskit.quantum_info import PTM, Kraus, Choi
from qiskit.quantum_info import state_fidelity
from qiskit import QuantumCircuit

from qiskit.extensions import HGate, XGate

In [3]:
def collect_tomography_data(shots=10000,
                            noise_model=None,
                            gateset_basis='Standard GST'):
    backend_qasm = Aer.get_backend('qasm_simulator')
    circuits = gateset_tomography_circuits(gateset_basis=gateset_basis)
    qobj = assemble(circuits, shots=shots)
    result = backend_qasm.run(qobj, noise_model=noise_model).result()
    fitter = GatesetTomographyFitter(result, circuits, gateset_basis)
    return fitter

def gate_set_tomography(gate, noise_model=None):
    basis = default_gateset_basis()
    basis.add_gate(gate)
    fitter = collect_tomography_data(shots=10000, noise_model=noise_model, gateset_basis=basis)
    result_gates = fitter.fit()
    result_gate = result_gates[gate.name]
    return result_gate

def print_results(gate, expected_gate):
    print("Expected result:")
    print(np.real(np.round(expected_gate,decimals=2)))
    print()
    print("Result:")
    print(np.real(np.round(gate,decimals=2)))
    print()
    distance = np.linalg.norm(np.array(expected_gate.data) - np.array(gate.data))
    print("Distance = ", distance)

### H gate, no noise

In [7]:
gate = gate_set_tomography(HGate())
expected_gate = PTM(HGate()).data
print_results(gate, expected_gate)

Expected result:
[[ 1.  0.  0.  0.]
 [ 0.  0.  0.  1.]
 [ 0.  0. -1.  0.]
 [ 0.  1.  0.  0.]]

Result:
[[ 1.01  0.01  0.    0.  ]
 [-0.01 -0.02 -0.    1.  ]
 [-0.    0.01 -1.   -0.01]
 [-0.    1.    0.01  0.  ]]

Distance =  0.03383507236934219


### X gate, no noise

In [9]:
gate = gate_set_tomography(XGate())
expected_gate = PTM(XGate()).data
print_results(gate, expected_gate)

Expected result:
[[ 1.  0.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  0. -1.  0.]
 [ 0.  0.  0. -1.]]

Result:
[[ 1.01  0.01  0.    0.  ]
 [-0.    0.98 -0.01  0.01]
 [ 0.    0.01 -0.98 -0.01]
 [-0.    0.    0.01 -0.99]]

Distance =  0.037759466859036216


### X gate, amplitude damping ($\gamma = 0.1$)

In [10]:
gamma = 0.1
A0 = [[1, 0], [0, np.sqrt(1 - gamma)]]
A1 = [[0, np.sqrt(gamma)], [0, 0]]
noise_ptm = PTM(Kraus([A0, A1])).data
noise = QuantumError([([{'name': 'kraus',
                         'qubits': [0],
                         'params': [A0, A1]}], 1)])
noise_model = NoiseModel()
noise_model.add_all_qubit_quantum_error(noise, ['u1', 'u2', 'u3', 'x'])

gate = gate_set_tomography(XGate(), noise_model)
expected_gate = noise_ptm @ PTM(XGate()).data
print_results(gate, expected_gate)

Expected result:
[[ 1.    0.    0.   -0.  ]
 [ 0.    0.95  0.    0.  ]
 [ 0.    0.   -0.95  0.  ]
 [ 0.1   0.    0.   -0.9 ]]

Result:
[[ 1.    0.02 -0.01  0.  ]
 [ 0.01  0.94  0.02 -0.  ]
 [ 0.02 -0.01 -0.95  0.02]
 [-0.    0.    0.   -0.9 ]]

Distance =  0.11220330983336729


### X gate, depolarization ($p=0.1$)

In [13]:
p = np.sqrt(0.1)
X = [[0, p], [p, 0]]
Y = [[0, -1j*p], [1j*p, 0]]
Z = [[p, 0], [0, -p]]
noise_ptm = PTM(Kraus([X,Y,Z])).data
noise = QuantumError([([{'name': 'kraus',
                         'qubits': [0],
                         'params': [X,Y,Z]}], 1)])
noise_model = NoiseModel()
noise_model.add_all_qubit_quantum_error(noise, ['u1', 'u2', 'u3', 'x'])

gate = gate_set_tomography(XGate(), noise_model)
expected_gate = noise_ptm @  PTM(XGate()).data
print_results(gate, expected_gate)

Expected result:
[[ 0.3  0.   0.   0. ]
 [ 0.  -0.1  0.   0. ]
 [ 0.   0.   0.1  0. ]
 [ 0.   0.   0.   0.1]]

Result:
[[ 0.69 -0.06 -0.04  0.23]
 [-0.12  0.13  0.08 -0.02]
 [-0.01  0.11 -0.47 -0.19]
 [-0.16  0.07 -0.25 -0.32]]

Distance =  0.9612696784586447


In [31]:
from qiskit.quantum_info import DensityMatrix, SuperOp
basis = default_gateset_basis()
basis.add_gate(HGate())
fitter = collect_tomography_data(shots=10000, gateset_basis=basis)
result_gates = fitter.fit()
rho = result_gates['rho']
E = result_gates['E']
I = np.array([[1,0],[0,1]])
X = np.array([[0,1],[1,0]])
Y = np.array([[0,-1j],[1j,0]])
Z = np.array([[1,0],[0,-1]])

rho_matrix = DensityMatrix(rho[0][0] * I +
                           rho[1][0] * X + 
                           rho[2][0] * Y + 
                           rho[3][0] * Z)

print(E)
E_matrix = Operator((E[0][0] * I +
                    E[0][1] * X + 
                    E[0][2] * Y + 
                    E[0][3] * Z) / 2)
E_matrix

[[1.+0.j 0.+0.j 0.+0.j 1.+0.j]]


Operator([[1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j]], input_dims=(2,), output_dims=(2,))

In [42]:
def _convert_from_ptm(vector):
    """Converts a vector back from PTM representation"""
    I = np.sqrt(0.5)*np.array([[1, 0], [0, 1]])
    X = np.sqrt(0.5)*np.array([[0, 1], [1, 0]])
    Y = np.sqrt(0.5)*np.array([[0, -1j], [1j, 0]])
    Z = np.sqrt(0.5)*np.array([[1, 0], [0, -1]])
    v = vector.reshape(4)
    return v[0] * I + v[1] * X + v[2] * Y + v[3] * Z
rho = np.array([[np.sqrt(0.5)], [0], [0], [np.sqrt(0.5)]])
_convert_from_ptm(rho)
_convert_from_ptm(E)

array([[1.41421356+0.j, 0.        +0.j],
       [0.        +0.j, 0.        +0.j]])

In [43]:
M = np.array([[1,2],[3,4]])
M.reshape(4)

array([1, 2, 3, 4])